In [6]:

import os
import shutil
from keras.backend import tensorflow_backend
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.utils import plot_model
from imutils import paths
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [7]:
dataDir = "DATA/animals"
trainPath = os.path.join(dataDir, "train")
validPath = os.path.join(dataDir, "valid")
testPath = os.path.join(dataDir, "test")

In [8]:
trainAugGen = ImageDataGenerator(rescale=1./255)

In [9]:
validAugGen = ImageDataGenerator(rescale=1./255)
testAugGen = ImageDataGenerator(rescale=1./255)

In [10]:
# Define some parameters of run
size = 150
batchSize = 32
learnRate = 0.001

In [11]:
trainGen = trainAugGen.flow_from_directory(
    trainPath,
    target_size=(size, size),
    batch_size=batchSize,
    class_mode='categorical')

Found 3500 images belonging to 3 classes.


In [12]:
validGen = validAugGen.flow_from_directory(
    validPath,
    target_size=(size, size),
    batch_size=batchSize,
    class_mode='categorical')

Found 1000 images belonging to 3 classes.


In [13]:
testGen = testAugGen.flow_from_directory(
    testPath,
    target_size=(size, size),
    batch_size=1,
    class_mode='categorical',
shuffle=False)

Found 500 images belonging to 3 classes.


In [14]:
# Determine the classes from the generator
classLabs = list(trainGen.class_indices.keys())
nClasses = len(classLabs)
print("[INFO] {:d} classes: {}".format(nClasses, classLabs))

[INFO] 3 classes: ['cat', 'dog', 'panda']


In [15]:
# Count the number of train, valid and test images
# ALT: trainImgPaths = list(paths.list_images(trainPath))
trainImgPaths = trainGen.filenames
nTrainImg = len(trainImgPaths)
validImgPaths = validGen.filenames
nValidImg = len(validImgPaths)
testImgPaths = testGen.filenames
nTestImg = len(testImgPaths)
print("[INFO] {:d} training images.".format(nTrainImg))
print("[INFO] {:d} validation images.".format(nValidImg))
print("[INFO] {:d} test images.".format(nTestImg))

[INFO] 3500 training images.
[INFO] 1000 validation images.
[INFO] 500 test images.


In [16]:
opt = SGD(lr=learnRate)    

In [17]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

#-----------------------------------------------------------------------------#
class SimpleNet:
    @staticmethod
    def build(width, height, depth, classes):

        # Initialize the model and channel order
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # First CONV(32) => RELU => POOL(2) layer set
        model.add(Conv2D(32, (3, 3), input_shape=inputShape))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Second CONV(32) => RELU => POOL(2) layer set
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Third CONV(64) => RELU => POOL(2) layer set
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # FC(64) => RELU => DROP(0.5)
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))

        # Softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        return model

In [18]:
model = SimpleNet.build(width=size, height=size,
                                 depth=3, classes=len(classLabs))

In [19]:
model.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])

In [20]:
from keras.callbacks import ModelCheckpoint

In [21]:
outDir = 'OUT'
if not os.path.exists(outDir):
        os.makedirs(outDir)

In [24]:
modelPath = os.path.join(outDir, "model_best.hdf5")

checkpoint = ModelCheckpoint(modelPath,                                   
                            monitor="val_loss",
                            save_best_only=True,
                            save_weights_only=False,
                            verbose=0)

callbacks = [checkpoint]

In [25]:
 H = model.fit_generator(
        trainGen,
        steps_per_epoch=nTrainImg / batchSize,
        validation_data=validGen,
        validation_steps=nValidImg / batchSize,
        epochs=100,
        callbacks=callbacks,
        verbose=1)

Epoch 1/100
110/109 [==============================] - 12s 110ms/step - loss: 1.0583 - acc: 0.3982 - val_loss: 1.0410 - val_acc: 0.4000
Epoch 2/100
110/109 [==============================] - 10s 88ms/step - loss: 1.0348 - acc: 0.4144 - val_loss: 1.0233 - val_acc: 0.4070
Epoch 3/100
110/109 [==============================] - 10s 87ms/step - loss: 1.0201 - acc: 0.4143 - val_loss: 1.0077 - val_acc: 0.4140
Epoch 4/100
110/109 [==============================] - 10s 87ms/step - loss: 1.0017 - acc: 0.4530 - val_loss: 0.9880 - val_acc: 0.4260
Epoch 5/100
110/109 [==============================] - 10s 88ms/step - loss: 0.9849 - acc: 0.4656 - val_loss: 0.9705 - val_acc: 0.4660
Epoch 6/100
110/109 [==============================] - 10s 87ms/step - loss: 0.9716 - acc: 0.4641 - val_loss: 0.9508 - val_acc: 0.5270
Epoch 7/100
110/109 [==============================] - 10s 88ms/step - loss: 0.9505 - acc: 0.4957 - val_loss: 0.9362 - val_acc: 0.5040
Epoch 8/100
110/109 [==============================] -

110/109 [==============================] - 11s 100ms/step - loss: 0.7006 - acc: 0.6578 - val_loss: 0.7299 - val_acc: 0.6300
Epoch 62/100
110/109 [==============================] - 11s 97ms/step - loss: 0.7094 - acc: 0.6541 - val_loss: 0.7126 - val_acc: 0.6450
Epoch 63/100
110/109 [==============================] - 11s 98ms/step - loss: 0.6911 - acc: 0.6633 - val_loss: 0.7329 - val_acc: 0.6350
Epoch 64/100
110/109 [==============================] - 11s 99ms/step - loss: 0.6885 - acc: 0.6691 - val_loss: 0.7064 - val_acc: 0.6410
Epoch 65/100
110/109 [==============================] - 11s 98ms/step - loss: 0.6975 - acc: 0.6530 - val_loss: 0.7150 - val_acc: 0.6390
Epoch 66/100
110/109 [==============================] - 10s 95ms/step - loss: 0.6979 - acc: 0.6533 - val_loss: 0.7413 - val_acc: 0.5980
Epoch 67/100
110/109 [==============================] - 11s 97ms/step - loss: 0.6826 - acc: 0.6649 - val_loss: 0.6995 - val_acc: 0.6630
Epoch 68/100
110/109 [==============================] - 11s 